In [ ]:
# import NumPy for linear algebra computations
import numpy as np
from numpy.linalg import matrix_power 

# import random for generating random numbers
import random

In [ ]:
# mathematical constant
SQRT_2 = np.sqrt(2)

# single qubit gates
I = np.eye(2)
H = (1/SQRT_2) * np.array([[1,1],[1,-1]])
Z = np.array([[1,0],[0,-1]])
X = np.array([[0,1],[1,0]])
P = np.array([[1,0],[0,1j]])
T = np.array([[1,0],[0, np.exp(1j * np.pi/4)]])

# two-qubit gates
CNOT = np.array([[1,0,0,0],[0,1,0,0],[0,0,0,1],[0,0,1,0]])
SWAP = np.array([[1,0,0,0],[0,0,1,0],[0,1,0,0],[0,0,0,1]])

In [ ]:
def gen_qubit(alpha, beta):
    """
    Generate a single qubit according to the probability amplitudes alpha and beta 
    and the constraint alpha^2 + beta^2 = 1.
    
    Args: 
        - alpha: a real number between 0 and 1 inclusive
        - beta: a real number between 0 and 1 inclusive
        
    Returns: 
        - the vector alpha |0> + beta |1>
    """
    return np.array([[alpha],[beta]]) # alpha^2 + beta^2 = 1

def zero_state():
    """
    Generate the zero state |0>.
    
    Returns:
        - the vector 1|0>
    """
    return gen_qubit(1,0)


def one_state():
    """
    Generate the one state |1>.
    
    Returns:
        - the vector 1|1>
    """
    
    return gen_qubit(0,1)


def superposition():
    """
    Generate the plus state |+> = 1/sqrt(2) * (|0> + |1>).
    
    Returns:
        - the vector 1/sqrt(2) * (|0> + |1>)
    """
    kat_zero = zero_state()
    return np.matmul(H, kat_zero)


def gen_epr():
    """
    Generate the EPR state 1/sqrt(2) * (|00> + |11>).
    
    Returns:
        - the vector 1/sqrt(2) * (|00> + |11>)
    """
    # generate two quantum registers and initialize them to the zero stae
    qubit_1, qubit_2 = zero_state(), zero_state()
    
    # apply the H gate to the first qubit
    qubit_1 = np.matmul(H, qubit_1)
    
    # apply the CNOT gate and obtain the EPR state
    epr = np.matmul(CNOT, np.kron(qubit_1, qubit_2))
    
    return epr

def pure_to_density(psi):
    """
    Compute the density matrix of a pure state |\psi>.
    
    Args: 
        - psi: the vector describing a pure state
        
    Returns:
        - |\psi> <\psi|
    """
    return np.matmul(psi, psi.conj().T)


def computational_basis(n: int):
    """
    Generate all the basis elements of an n-dimension qubit system in the computational basis
    For example, if n=1, the function will return the set {|0>,|1>} as a numpy array.
    
    Args:
        - n: the dimension of the system
        
    Returns:
        - arrays: all the basis elements of the n-dimensional qubit system in the computational basis  
    """
    # create 2^n zero arrays
    arrays = [np.zeros((2**n, 1)) for i in range(2**n)]
    
    # substitute 1 for the i^th element of the i^th array
    for i in range(2**n):
        arrays[i][i] = 1
        
    return arrays

def measurement(rho): # measurement is performed in the computational basis
    """
    Return a random string of bits upon the measurement of the density matrix \rho
    according to projections onto the computational basis.
    
    Args:
        - rho: density matrix of a quantum state
        
    Returns:
        - classical_bits: a string of bits with the same length as the dimension of the quantum state \rho
    """
    # compute the dimension of the system
    dimension = int(np.log2(rho.shape[0]))
    
    # generate all the basis elements
    computational_bases = computational_basis(dimension)
    
    # initialize a list with 2^n elements with zero
    # which stores the probability amplitudes that 
    # result from the projections onto the computational basis
    probabilities = [0]*(2**dimension)
    
    for i in range(2**dimension):
        probabilities[i] = np.abs(np.trace(np.matmul(rho, pure_to_density(computational_bases[i]))))
    
    # randomly make a choice based on the probability amplitudes
    measurement_res = np.random.choice(len(probabilities), p = probabilities)
    
    classical_bits = bin(measurement_res)[2:].zfill(dimension)
    return classical_bits


def operator_builder(string, L):
    """
    Concatenate a string of "I"s whose positions are specified by the list L with the string variable.
    For example, operator_builder("01", [0,2]) returns the string "I0I1".
    This function will be useful for computing the partial trace of a quantum state.

    Args: 
        - string: a binary string
        - L: a list
        
    Returns: 
        - the string "I" at positions specified by the list L concatenated with the string
    """
    counter = 0
    
    # total length of the string
    n = len(string) + len(L)
    
    # initalize a 0 string as a list
    new_string = list("0"*n)
    
    # set the L[i] location of the new string to "I"
    for i in range(len(L)):
        new_string[L[i]] = "I"
        
    # set the remaining positions according to the string variable that was passed by as an argument
    for j in range(n):
        if new_string[j] == "I":
            pass
        else:
            new_string[j] = string[counter]
            counter = counter + 1
    
    # turn the list into a string variable
    return "".join(new_string)


def partial_trace(rho, L):
    """
    Compute the partial trace of a density matrix rho, while keeping the registers specified in the list L
    and tracing out registers that are not in L.
    
    Args:
        - rho: density matrix of a quantum state
        - L: registers to keep
        
    Returns: 
        - rho_density: partial trace of rho
    """
    system_dimension = int(np.log2(rho.shape[0]))
    trace_out_dimension = system_dimension - len(L)    
    rho_density = np.zeros((2**len(L), 2**len(L)))
    
    for i in range(2**trace_out_dimension):
        operator_matrix = np.array(1)
        operator = bin(i)[2:].zfill(trace_out_dimension)
        operator = operator_builder(operator, L)

        for j in operator:
            if j == "0":
                operator_matrix = np.kron(operator_matrix, zero_state())
            if j == "1":
                operator_matrix = np.kron(operator_matrix, one_state())
            if j == "I":
                operator_matrix = np.kron(operator_matrix, I)   

        rho_density = rho_density + np.matmul(np.matmul(operator_matrix.conj().T, rho), operator_matrix)
        
    return rho_density

def qgate_on_density(quantum_gate, rho):
    """
    Applies a quantum gate on density matrix.
    
    Args: 
        - quantum_gate: a numpy array as the matrix representation of a quantum gate
        - rho: density matrix of a quantum state
        
    Returns:
        - quantum gate * rho * tranpose(conjugate(quantum gate))
    """
    return np.matmul(np.matmul(quantum_gate, rho), quantum_gate.conj().T)

def post_measurement_state(rho, L, bits): 
    """
    Given a density matrix rho and the results of a measurement of a sub-system of rho, 
    return the post-measurement state.
    
    Args: 
        - rho: density matrix of a quantum state
        - L: indices of sub-system(s) of rho that are measured as a list
        bits: a binary string indicating the results of the measured sub-system(s)
        
    Returns:
        - rho_pms: the density matrix of the post-measurement state
        - bits: the same as the input of the matrix
    """
    system_dimension = int(np.log2(rho.shape[0]))
    operator = list("I"*system_dimension)
    for i in range(len(L)):
        operator[L[i]] = bits[i]
    operator = ''.join(i for i in operator)
    projection_operator = np.array(1)
    zero_projection = np.kron(zero_state(), zero_state().conj().T)
    one_projection = np.kron(one_state(), one_state().conj().T)
    
    for i in operator:
        if i == "0":
            projection_operator = np.kron(projection_operator, zero_projection)
            
        if i == "1":
            projection_operator = np.kron(projection_operator, one_projection)
            
        if i == "I":
            projection_operator = np.kron(projection_operator, I)
    
    rho_pms = (projection_operator @ rho @ projection_operator.conj().T) / (np.trace(projection_operator.conj().T @ projection_operator @ rho))
 
    return (rho_pms, bits)